# Match CAM and Save Image - Single Folder

### Import Package

In [1]:
## Package
from PIL import Image
from datetime import datetime
import random, os, sys, glob
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
module_path = '/home/sasdemo01'
if module_path not in sys.path:
    sys.path.append(module_path)
import util
from util.preprocess import *

### Setting Parameters

In [2]:
## Data name
data_name = "S1_5143_T_SR1+DAM_OSP+SOP_01_01"

## CAM File (.bmp)
cam = Image.open('/home/sasdemo01/data/AVI/S1/CAM/5143b69c01-uts+1_CompSideUnit.bmp')

## Training Folder (Folder)
Img_path = "/home/sasdemo01/data/AVI/S1/Training/"+data_name +"/"

## Training Converted (Folder)
Converted_path = "/home/sasdemo01/data/AVI/S1/Training_Convert/"+data_name+"/"

## Unit Position (.csv)
mapcsv = pd.read_csv('/home/sasdemo01/data/AVI/S1/Training/'+data_name+'_對應位置.csv')

In [3]:
## Set the numerical parameters

### Scale => um/pixel
scale = 4

## X_offset => mm
## Y_offset => mm
X_offset = 0
Y_offset = 0

### CAM size(1 Unit) => mm
camsize = 16.509


### Rotate CAM
cam = np.array(cam.rotate(180))
max_y = cam.shape[0]
max_x = cam.shape[1]

## Count the training data
aviimagedir, dirs, files = next(os.walk(Img_path+"NG"))
NG_count = len(files)
aviimagedir, dirs, files = next(os.walk(Img_path+"OK"))
OK_count = len(files)

print("OK: ", OK_count)
print("NG: ", NG_count)

OK:  3092
NG:  4323


## Convert and Save Image

In [7]:
## Convert all image
if not os.path.exists(Converted_path):
    os.mkdir(Converted_path)
    os.mkdir(Converted_path+'NG/')
    os.mkdir(Converted_path+'OK/')
li = (range(mapcsv.shape[0]-1))
for n in li:
    if(n%100==0):
        print("Image: ",n," / ",mapcsv.shape[0]-1)
        print(datetime.now())
    fold = 'NG'

    im = cv.imread(Img_path+"NG/" + mapcsv.image_path[n])
    if(type(im)==type(None)):
        fold='OK'
        im = cv.imread(Img_path+"OK/" + mapcsv.image_path[n])
        

    ## Resize the avi image
    size1 = int(max_x*(scale*im.shape[1]/1000)/(camsize))
    size2 = int(max_y*(scale*im.shape[0]/1000)/(camsize))
    im = cv.resize(im,(size1,size2),interpolation = cv.INTER_CUBIC)

    ## defect (x,y) on gbr file
    y_o = int(((mapcsv.Unit_Yvalue[n]-Y_offset)/camsize)*max_y)
    x_o = int(((mapcsv.Unit_Xvalue[n]-X_offset)/camsize)*max_x)

    ## Crop the gbr image
    cam_im_1x_lefttop= (cam[y_o:y_o+int(size2),x_o:x_o+int(size1)])
    cam_im_15x_lefttop= (cam[max(y_o-int(size2*0.25),0):min(max_y,y_o+int(size2*1.25)),max(0,x_o-int(size1*0.25)):min(max_x,x_o+int(size1*1.25))])



    ## Preprocess image
    img = []
    img.append(im[:,:,0])
    img.append(BW_1(im[:,:,0]))
    #img.append(BW_2(im))
    img.append(Grey_1(im[:,:,0]))
    img.append(Grey_2(im[:,:,0]))
    img.append(Grey_3(im[:,:,0]))
    ## Select which size to match
    cam_to_match = cam_im_15x_lefttop.copy()
    ## Match Template
    res = []
    pos = []
    sim = []
    Flag = True
    for i in range(len(img)):
        try:
            res.append(cv.matchTemplate(img[i], np.array(cam_to_match), cv.TM_CCOEFF_NORMED))
            pos.append(np.where(res[i]>=res[i].max()))
            sim.append(res[i].max())
            Flag=True
        except:
            Flag=False
    if(Flag):
        loc_most_alike = pos[np.argmax(sim)]
        index = np.argmax(sim)

        x_min = loc_most_alike[1][0].item()
        y_min = loc_most_alike[0][0].item()
        img_targ = np.array(cam_to_match)[y_min:y_min+im.shape[0],x_min:x_min+im.shape[1]]
        if(img_targ.shape!=im[:,:,0].shape):
            img_targ=im[:,:,0].copy()
    else:
        print(n)
        img_targ = im[:,:,0].copy()
        index = 0
    im_new = im.copy()

    im_twoch = im_new.copy()
    im_twoch[:,:,2] = np.zeros([im.shape[0],im.shape[1]])
    im_twoch[:,:,1] = img_targ*0.5
    Image.fromarray(im_twoch).save(Converted_path + fold+'/'+mapcsv.image_path[n])        

Image:  0  /  7414
2020-02-12 15:07:51.922538
Image:  100  /  7414
2020-02-12 15:08:03.230993
Image:  200  /  7414
2020-02-12 15:08:14.468583
Image:  300  /  7414
2020-02-12 15:08:25.480057
Image:  400  /  7414
2020-02-12 15:08:34.916821
Image:  500  /  7414
2020-02-12 15:08:45.149544
Image:  600  /  7414
2020-02-12 15:09:01.500297
Image:  700  /  7414
2020-02-12 15:09:14.950407
Image:  800  /  7414
2020-02-12 15:09:28.863721
Image:  900  /  7414
2020-02-12 15:09:43.871654
Image:  1000  /  7414
2020-02-12 15:09:58.008134
Image:  1100  /  7414
2020-02-12 15:10:12.380816
Image:  1200  /  7414
2020-02-12 15:10:26.705925
Image:  1300  /  7414
2020-02-12 15:10:40.506559
Image:  1400  /  7414
2020-02-12 15:10:55.077344
Image:  1500  /  7414
2020-02-12 15:11:09.364658
Image:  1600  /  7414
2020-02-12 15:11:23.967643
Image:  1700  /  7414
2020-02-12 15:11:34.397847
Image:  1800  /  7414
2020-02-12 15:11:44.914454
Image:  1900  /  7414
2020-02-12 15:11:54.589388
Image:  2000  /  7414
2020-02-12